# Случайный лес (Random Forest)

**Выполнила:** Медведева Ольга Игоревна

**Вариант:** 2

**Данные:** default of credit card clients

**Задание:** 
- Построить модель для распознавания целевой переменной (отклика)
- Для распознавания использовать Случайный лес (Random Forest)
- Выбрать критерии качества
- Обосновать выбор параметров модели

In [1]:
# Загружаю бибилотеки

import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
# Импортирую данные

df = pd.read_excel('default of credit card clients.xls', index_col=0, skiprows=1)

df.head(10)

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
ID,,,,,,,,,,,,,,,,,,,,,
1,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
2,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
5,50000,1,2,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0
6,50000,1,1,2,37,0,0,0,0,0,...,19394,19619,20024,2500,1815,657,1000,1000,800,0
7,500000,1,1,2,29,0,0,0,0,0,...,542653,483003,473944,55000,40000,38000,20239,13750,13770,0
8,100000,2,2,2,23,0,-1,-1,0,0,...,221,-159,567,380,601,0,581,1687,1542,0
9,140000,2,3,1,28,0,0,2,0,0,...,12211,11793,3719,3329,0,432,1000,1000,1000,0


In [3]:
# Считаю, сколько пропусков в каждом столбце

NAs = pd.concat([df.isnull().sum()], axis=1)
NAs[NAs.sum(axis=1) > 0]
NAs

,0
LIMIT_BAL,0
SEX,0
EDUCATION,0
MARRIAGE,0
AGE,0
PAY_0,0
PAY_2,0
PAY_3,0
PAY_4,0
PAY_5,0


Видно что таблица содержит 0 в столбцах X6 - X11 (PAY_0 	PAY_2 	PAY_3 	PAY_4 	PAY_5, PAY_6) с историей последних платежей, где возможные значения -1, 1, 2 и т.д в зависимости от срока выплаты. Очень важный параметр для принятия решения о выдаче кредита в банке и неверные значения могут сильно исказить результат. Поэтому попробую убрать из таблицы строки, в которых все эти значения одновременно равны нулю.

In [4]:
df = df.loc[(df['PAY_0'] != 0) & (df['PAY_2'] != 0) & (df['PAY_3'] != 0) & (df['PAY_4'] != 0) & (df['PAY_5'] != 0) & (df['PAY_6'] != 0)]

In [5]:
# Проверяю размерность матрицы данных

df.shape

(8827, 24)

In [6]:
df.describe(include='all')

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
count,8827.000000,8827.000000,8827.000000,8827.000000,8827.000000,8827.000000,8827.000000,8827.000000,8827.000000,8827.000000,...,8827.000000,8827.000000,8827.000000,8827.000000,8.827000e+03,8827.00000,8827.000000,8827.000000,8827.000000,8827.000000
mean,199871.983686,1.632151,1.708395,1.494279,36.706356,-0.403308,-0.720177,-0.740569,-0.799366,-0.834372,...,12527.659227,12292.814206,12582.552056,4152.849892,4.304666e+03,3990.07579,3850.464937,4250.515011,4377.829500,0.270647
std,128347.916927,0.482247,0.786630,0.517588,9.001289,1.558229,1.614321,1.652577,1.658182,1.629724,...,34028.097918,33266.388712,34404.763585,12310.734570,2.202718e+04,13024.38371,11689.467561,14950.612411,15868.897998,0.444319
min,10000.000000,1.000000,0.000000,0.000000,21.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,...,-65167.000000,-53007.000000,-339603.000000,0.000000,0.000000e+00,0.00000,0.000000,0.000000,0.000000,0.000000
25%,100000.000000,1.000000,1.000000,1.000000,30.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.00000,0.000000,0.000000,0.000000,0.000000
50%,190000.000000,2.000000,2.000000,1.000000,35.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,1006.000000,952.000000,986.000000,770.000000,6.410000e+02,527.00000,530.000000,477.000000,423.000000,0.000000
75%,280000.000000,2.000000,2.000000,2.000000,42.000000,1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,7742.500000,7501.500000,7670.500000,3500.000000,3.384000e+03,3063.00000,2915.000000,3000.000000,2956.000000,1.000000
max,800000.000000,2.000000,6.000000,3.000000,75.000000,8.000000,8.000000,8.000000,8.000000,8.000000,...,486776.000000,503914.000000,527711.000000,298887.000000,1.684259e+06,417588.00000,265852.000000,379267.000000,403500.000000,1.000000


In [7]:
# Классы сбалансированы?

df['default payment next month'].value_counts()

0    6438
1    2389
Name: default payment next month, dtype: int64

Количество экземпляров в классе 0 превышает количество экземпляров в классе 1 в 2,7 раза. Это много, но данных стало меньше после очистки, поэтому будем работать с таким распределением данных по классам.

In [8]:
df.head(10)

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
ID,,,,,,,,,,,,,,,,,,,,,
1,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
10,20000,1,3,2,35,-2,-2,-2,-2,-1,...,0,13007,13912,0,0,0,13007,1122,0,0
12,260000,2,1,2,51,-1,-1,-1,-1,-1,...,8517,22287,13668,21818,9966,8583,22301,0,3640,0
19,360000,2,1,1,49,1,-2,-2,-2,-2,...,0,0,0,0,0,0,0,0,0,0
20,180000,2,1,2,29,1,-2,-2,-2,-2,...,0,0,0,0,0,0,0,0,0,0
22,120000,2,2,1,39,-1,-1,-1,-1,-1,...,0,632,316,316,316,0,632,316,0,1
24,450000,2,1,1,40,-2,-2,-2,-2,-2,...,560,0,0,19428,1473,560,0,0,1128,1
27,60000,1,1,2,27,1,-2,-1,-1,-1,...,-57,127,-189,0,1000,0,500,0,1000,1
29,50000,2,3,1,47,-1,-1,-1,-1,-1,...,2040,30430,257,3415,3421,2044,30430,257,0,0


In [9]:
# Разделяю предикоторы и отклики

X = df.iloc[:, :-1].values  
y = df.iloc[:, 23].values 

In [10]:
# Проверяю себя

y

array([1, 0, 0, ..., 0, 1, 1], dtype=int64)

In [11]:
# Разделяю на обучающую и тестовую выборку

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state = 1234)

In [12]:
# Задаю начальные параметры модели
# Инициализирую модель

model = RandomForestClassifier(random_state=42,
                               # число деревьев в лесу
                               n_estimators=80,
                               # функция для impurity ('gini' или 'entropy')
                               criterion='entropy',
                               # максимальное число слоев
                               max_depth=5,
                               # сколько переменных использовать при построении дерева
                               max_features='auto', 
                               # максимальное число конечных узлов
                               max_leaf_nodes=None,
                               max_samples=0.66,
                               # минимальное уменьшение загрязнения 
                               min_impurity_decrease=0.001, 
                               # дерево обучается по части наблюдений
                               bootstrap=True, 
                               # минимальное число наблюдений в узле потомке
                               min_samples_leaf=8, 
                               # минимальное число наблюдений в узле родителе
                               min_samples_split=12,
                               # параметр используется редко, самостоятельно
                               min_weight_fraction_leaf=0.0, 
                               # объем промежуточных результатов в ходе обучения
                               verbose=1, 
                               # число используемых процессоров
                               # n_jobs=None,
                               # Вычислять out-of-bag ошибку
                               oob_score=True,
                               # использовать результаты предыдущего вызова и нарастить предыдущий лес 
                               warm_start=False,
                               # веса классов для балансировки обучения
                               class_weight=None)

# Обучаю модель

model.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    1.0s finished


RandomForestClassifier(criterion='entropy', max_depth=5, max_samples=0.66,
                       min_impurity_decrease=0.001, min_samples_leaf=8,
                       min_samples_split=12, n_estimators=80, oob_score=True,
                       random_state=42, verbose=1)

In [13]:
# Распознаю классы, используя модель

y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    0.0s finished


In [14]:
# Предсказание класса для новых элементов

new_item = [6000, 1, 2, 1, 45, -1, 1, -1, -1, 1, 2, 4000, 3800, 3500, 2700, 2220, 1375, 1578, 1423, 1287, 1000, 980, 620]
model.predict([new_item])

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    0.0s finished


array([0], dtype=int64)

In [15]:
# Считаю accuracy модели

print("Accuracy on train set is ", accuracy_score(y_train, y_pred_train)*100)
print("Accuracy on test set is ", accuracy_score(y_test, y_pred_test)*100)

Accuracy on train set is  79.42171119377748
Accuracy on test set is  79.3683487813251


In [16]:
print(metrics.classification_report(y_pred_test, y_test))

              precision    recall  f1-score   support

           0       0.93      0.81      0.87      2438
           1       0.42      0.71      0.53       475

    accuracy                           0.79      2913
   macro avg       0.68      0.76      0.70      2913
weighted avg       0.85      0.79      0.81      2913



In [17]:
print(metrics.precision_recall_fscore_support(y_pred_test, y_test, average='weighted'))

(0.8505911788599082, 0.7936834878132509, 0.8125396933649999, None)


В метриках качества посмотрим на взвешенное среднее по классам для precision, recall, f1-score, так как число экземпляров в наших классах отличается.

Воспользуюсь методом feature_importances_ для оценки ценности каждого параметра данных в таблице.

In [18]:
pd.DataFrame({'feature': df.iloc[:, :-1].columns,
              'importance': model.feature_importances_}).sort_values('importance', ascending=False)

,feature,importance
8,PAY_4,0.128254
6,PAY_2,0.127274
5,PAY_0,0.110993
9,PAY_5,0.107837
10,PAY_6,0.089916
7,PAY_3,0.053953
14,BILL_AMT4,0.041159
16,BILL_AMT6,0.034835
13,BILL_AMT3,0.034267
17,PAY_AMT1,0.033677


Самыми ценными параметрами (топ-6) являются платежи из истории платежей по кредиту. От 13% до 6%. Случайный лес в отличии от CART ни одному параметру не дает нулевую ценность.

In [19]:
# Рассмотрю матрицы распределения по классам для обучающей выборки

conf_mat = metrics.confusion_matrix(y_train, y_pred_train)
conf_mat = pd.DataFrame(conf_mat, index=model.classes_, columns=model.classes_)
conf_mat

,0,1
0,4032,291
1,926,665


In [20]:
# Рассмотрю матрицы распределения по классам для тестовой выборки

conf_mat = metrics.confusion_matrix(y_test, y_pred_test)
conf_mat = pd.DataFrame(conf_mat, index=model.classes_, columns=model.classes_)
conf_mat

,0,1
0,1976,139
1,462,336


In [21]:
#  С помощью модели оцениваем вероятности принадлежать классам 

y_pred_train2 = model.predict_proba(X_train)
y_pred_test2 = model.predict_proba(X_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    0.0s finished


In [22]:
print(y_pred_train2)
print(y_pred_test2)

[[0.78143823 0.21856177]
 [0.86824186 0.13175814]
 [0.26813111 0.73186889]
 ...
 [0.92139657 0.07860343]
 [0.83331633 0.16668367]
 [0.7433811  0.2566189 ]]
[[0.82950026 0.17049974]
 [0.86283856 0.13716144]
 [0.75208136 0.24791864]
 ...
 [0.7635209  0.2364791 ]
 [0.90281634 0.09718366]
 [0.69329674 0.30670326]]


## Подбор внешних параметров с помощью GridSearch

**Выбор параметров**

Результаты обучения модели показали отсутствие переобучения. Следовательно, можно двигаться в сторону углубления модели. Увеличить max_depth и уменьшить параметры min_samples_leaf и min_samples_split.

In [23]:
#  Определяю интересные значения гиперпараметров

param_grid = {  
'n_estimators': [30, 40, 50, 80, 100],
'max_depth': [5, 6, 7, 8],
'min_samples_leaf': [4, 6, 8, 10],
'min_samples_split': [8, 10, 12, 15]
}

In [24]:
# Инициализирую модель

model2 = RandomForestClassifier(random_state=42,
                               # число деревьев в лесу
                               n_estimators=80,
                               # функция для impurity ('gini' или 'entropy')
                               criterion='entropy',
                               # максимальное число слоев
                               max_depth=5,
                               # сколько переменных использовать при построении дерева
                               max_features='auto', 
                               # максимальное число конечных узлов
                               max_leaf_nodes=None,
                               max_samples=0.66,
                               # минимальное уменьшение загрязнения 
                               min_impurity_decrease=0.001, 
                               # дерево обучается по части наблюдений
                               bootstrap=True, 
                               # минимальное число наблюдений в узле потомке
                               min_samples_leaf=8, 
                               # минимальное число наблюдений в узле родителе
                               min_samples_split=12,
                               # параметр используется редко, самостоятельно
                               min_weight_fraction_leaf=0.0, 
                               # объем промежуточных результатов в ходе обучения
                               verbose=1, 
                               # число используемых процессоров
                               # n_jobs=None,
                               # Вычислять out-of-bag ошибку
                               oob_score=True,
                               # использовать результаты предыдущего вызова и нарастить предыдущий лес 
                               warm_start=False,
                               # веса классов для балансировки обучения
                               class_weight=None)


grid_search_res = GridSearchCV(model2, param_grid=param_grid, cv=2, 
                               #  выбор критерия качества
                               scoring='accuracy', 
                               #  поведение в случае ошибки при вычислении критерия качества
                               error_score='raise', 
                               #  обучить модель с лучшими значениями параметроы
                               refit=True, 
                               pre_dispatch=None, verbose=0)

In [25]:
grid_search_res.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Us

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.7s finished
[Parallel(n_jobs=1)]: Us

[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    0.6s finished
[Parallel(n_jobs=1)]: Us

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Us

...

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.6s finished
[Parallel(n_jobs=1)]: Us

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    0.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    0.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    0.9s finished
[Parallel(n_jobs=1)]: Us

GridSearchCV(cv=2, error_score='raise',
             estimator=RandomForestClassifier(criterion='entropy', max_depth=5,
                                              max_samples=0.66,
                                              min_impurity_decrease=0.001,
                                              min_samples_leaf=8,
                                              min_samples_split=12,
                                              n_estimators=80, oob_score=True,
                                              random_state=42, verbose=1),
             param_grid={'max_depth': [5, 6, 7, 8],
                         'min_samples_leaf': [4, 6, 8, 10],
                         'min_samples_split': [8, 10, 12, 15],
                         'n_estimators': [30, 40, 50, 80, 100]},
             pre_dispatch=None, scoring='accuracy')

In [26]:
#  Лучший набор значений гиперпараметров

best_hyperparams = grid_search_res.best_params_
print('Лучшие значения гиперпараметров:\n', best_hyperparams)

Лучшие значения гиперпараметров:
 {'max_depth': 7, 'min_samples_leaf': 6, 'min_samples_split': 8, 'n_estimators': 80}


In [27]:
# Значение критерия качества для лучших значений гиперпараметров

best_CV_score = grid_search_res.best_score_
print('Лучшее accuracy', best_CV_score)

Лучшее accuracy 0.7935407507609064


In [28]:
# Лучшая модель

best_model = grid_search_res.best_estimator_

# accuracy на тестовом множестве для лучшей модели

test_acc = best_model.score(X_test,y_test)

# Значение критерия качества для лучших значений гиперпараметров

print('Лучшее accuracy на тестовом множестве', test_acc)

Лучшее accuracy на тестовом множестве 0.7940267765190525


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    0.0s finished


In [29]:
means = grid_search_res.cv_results_['mean_test_score']
stds  = grid_search_res.cv_results_['std_test_score']

In [30]:
for mean, std, params in zip(means, stds, grid_search_res.cv_results_['params']):
    print("%0.4f (+/-%0.03f) for %r"  % (mean, std * 2, params))
print()

0.7925 (+/-0.013) for {'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 8, 'n_estimators': 30}
0.7915 (+/-0.014) for {'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 8, 'n_estimators': 40}
0.7902 (+/-0.013) for {'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 8, 'n_estimators': 50}
0.7917 (+/-0.016) for {'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 8, 'n_estimators': 80}
0.7917 (+/-0.014) for {'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 8, 'n_estimators': 100}
0.7907 (+/-0.016) for {'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 30}
0.7908 (+/-0.014) for {'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 40}
0.7910 (+/-0.015) for {'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 50}
0.7913 (+/-0.016) for {'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 80}
0.7915 (+/-0.015) for {'max_depth': 5, 'm

**Вывод:**
- Повысить accuracy модели за счет изменения гиперпараметров удалось с 79,368% до 79,403% на тестовом множестве.
- Модель не переобучается так как отличие между тренировочным и тестовым множеством менее 1%.
- В итоге accuracy на обучающей выборке равна 79,35%. На тренировочной - 79,40%, что лучше распознавания методом k-го ближайшего соседа и методом CART.